## T6 (ROMD) V. Roizman, M. Jonckheere and F. Pascal, "Robust clustering and outlier rejection using the Mahalanobis distance distribution," 2020 28th European Signal Processing Conference, Amsterdam, Netherlands, 2021, pp. 2448-2452, doi: 10.23919/Eusipco47968.2020.9287356.


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, silhouette_score, davies_bouldin_score
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import mahalanobis

# Load and preprocess the Iris dataset
# data = datasets.load_iris()
# X = data.data
# y = data.target
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

file_path = r"D:\Research Related\30 Deep Robust Clustering with Mahalanobis Distance (DRCMD)\dataset\anemia.csv"

# Load the dataset into a pandas DataFrame
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
print(data.head())

# Remove the last column
# Separate the features (X) and the target variable (y_true)
X = data.iloc[:, :-1]  # All columns except the last
y_true = data.iloc[:, -1]  # The last column
# # If all columns are features:
# X = data

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Parameters
n_clusters = 3  # Number of clusters (assume we know it for the Iris dataset)
alpha = 0.95  # Significance level for outlier rejection

# Robust Mahalanobis distance computation using Tyler's estimators
def robust_mahalanobis_distance(X, mean, cov):
    inv_cov = np.linalg.inv(cov)
    distances = [mahalanobis(x, mean, inv_cov) for x in X]
    return np.array(distances)

# Fit Gaussian Mixture Model (GMM) for clustering
gmm = GaussianMixture(n_components=n_clusters, covariance_type='full', random_state=42)
gmm.fit(X_scaled)
labels = gmm.predict(X_scaled)
means = gmm.means_
covariances = gmm.covariances_

# Compute Mahalanobis distances for each cluster
outliers = []
outlier_scores = np.zeros(X_scaled.shape[0])
for cluster in range(n_clusters):
    cluster_points = X_scaled[labels == cluster]
    mean = means[cluster]
    cov = covariances[cluster]
    distances = robust_mahalanobis_distance(cluster_points, mean, cov)
    
    # Chi-squared threshold for outlier detection
    threshold = np.percentile(distances, 100 * alpha)
    cluster_outliers = cluster_points[distances > threshold]
    outliers.extend(cluster_outliers)
    outlier_scores[labels == cluster] = distances

outliers = np.array(outliers)

# Metrics
silhouette = silhouette_score(X_scaled, labels)
davies_bouldin = davies_bouldin_score(X_scaled, labels)
s_db_ratio = silhouette / davies_bouldin if davies_bouldin != 0 else None

# Calculate AUC
true_labels = (y_true == 2).astype(int)  # Assume one class as "outliers"
fpr, tpr, _ = roc_curve(true_labels, outlier_scores)
roc_auc = auc(fpr, tpr)

# Print metrics
print("Silhouette Score:", silhouette)
print("Davies-Bouldin Index:", davies_bouldin)
print("S/DB Ratio:", s_db_ratio)
print(f"Number of Outliers Detected: {len(outliers)}")
print(f"Percentage of Outliers Detected: {(len(outliers) / len(X_scaled)) * 100:.2f}%")
print("AUC Value:", roc_auc)



# Visualizations
# Highlight Detected Outliers
plt.figure(figsize=(8, 6))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis', alpha=0.6, label='Inliers')
if len(outliers) > 0:
    plt.scatter(outliers[:, 0], outliers[:, 1], c='red', label='Outliers')
plt.title('Robust Clustering and Outlier Detection (Iris Dataset)')
plt.xlabel('Feature 1 (Standardized)')
plt.ylabel('Feature 2 (Standardized)')
plt.legend()
plt.show()

# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Research Related\\30 Deep Robust Clustering with Mahalanobis Distance (DRCMD)\\dataset\\anemia.csv'

In [ ]:
# 17/12/2024

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, roc_curve, auc, 
    adjusted_rand_score, f1_score
)
from scipy.spatial.distance import mahalanobis
from sklearn.neighbors import NearestNeighbors

# Load the dataset
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\pima.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\parkinsons.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\heart.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\hepatitis.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\ionosphere.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\anemia.csv"
#file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\syn1.csv"
file_path = r"C:\Users\psbis\2024 Clustering Autoencoder Mahalobish distance\syn2.csv"

data = pd.read_csv(file_path)

# Separate features and target
X = data.iloc[:, :-1]  # Features
y_true = data.iloc[:, -1]  # True labels for evaluation

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Parameters
n_clusters = 3  # Number of clusters
alpha = 0.95  # Confidence threshold for outlier detection

# Robust Mahalanobis distance computation
def robust_mahalanobis_distance(X, mean, cov):
    inv_cov = np.linalg.inv(cov + 1e-6 * np.eye(len(cov)))  # Regularize covariance
    distances = [mahalanobis(x, mean, inv_cov) for x in X]
    return np.array(distances)

# Fit Gaussian Mixture Model (GMM)
gmm = GaussianMixture(n_components=n_clusters, covariance_type='full', random_state=42)
gmm.fit(X_scaled)
labels = gmm.predict(X_scaled)
means = gmm.means_
covariances = gmm.covariances_

# Identify outliers using Mahalanobis distance
outliers = []
outlier_scores = np.zeros(X_scaled.shape[0])
for cluster in range(n_clusters):
    cluster_points = X_scaled[labels == cluster]
    mean = means[cluster]
    cov = covariances[cluster]
    distances = robust_mahalanobis_distance(cluster_points, mean, cov)
    
    # Threshold for outlier detection
    threshold = np.percentile(distances, 100 * alpha)
    cluster_outliers = cluster_points[distances > threshold]
    outliers.extend(cluster_outliers)
    outlier_scores[labels == cluster] = distances

outliers = np.array(outliers)

# Metrics: Silhouette, Davies-Bouldin, S/DB Ratio
silhouette = silhouette_score(X_scaled, labels)
davies_bouldin = davies_bouldin_score(X_scaled, labels)
s_db_ratio = silhouette / (davies_bouldin + 1e-6)

# Adjusted Rand Index (ARI)
ari = adjusted_rand_score(y_true, labels)

# F1 Score: Treat outliers as a binary classification problem
binary_labels = np.zeros_like(labels)
# F1 Score: Treat outliers as a binary classification problem
binary_labels = np.zeros(len(X_scaled), dtype=int)  # Initialize all as inliers

# Identify outlier indices by comparing rows in X_scaled with outliers
outlier_indices = [i for i, x in enumerate(X_scaled) if any(np.all(x == outlier) for outlier in outliers)]

# Update binary labels for identified outliers
binary_labels[outlier_indices] = 1

# Calculate F1 Score
from sklearn.metrics import f1_score
f1 = f1_score((y_true == 1).astype(int), binary_labels, average='weighted')

f1 = f1_score((y_true == 1).astype(int), binary_labels, average='weighted')

# Hubness Score
def hubness_score(X, k=5):
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    indices = nbrs.kneighbors(X, return_distance=False)
    hubness = np.bincount(indices.flatten(), minlength=len(X))
    return np.mean(hubness), np.std(hubness)

hubness_mean, hubness_std = hubness_score(X_scaled)

# AUC for anomaly scores
fpr, tpr, _ = roc_curve((y_true == 1).astype(int), outlier_scores)
roc_auc = auc(fpr, tpr)

# Print metrics
print("Silhouette Score:", np.round(silhouette,3))
print("Davies-Bouldin Index:", np.round(davies_bouldin,3))
print("S/DB Ratio:", np.round(s_db_ratio,3))
print("Adjusted Rand Index (ARI):", np.round(ari,3))
print("F1 Score:", np.round(f1,3))
print("Hubness Mean:", np.round(hubness_mean,3))
print("Hubness Std Dev:", np.round(hubness_std,3))
print("Number of Outliers:", np.round(outliers_count,3))
print(f"Percentage of Outliers: {outliers_percentage:.2f}%")
print("AUC Value:", roc_auc)

# Visualization: Outliers and Clusters
plt.figure(figsize=(8, 6))
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis', alpha=0.6, label="Inliers")
if len(outliers) > 0:
    plt.scatter(outliers[:, 0], outliers[:, 1], c='red', label='Outliers')
plt.title('Robust Clustering and Outlier Detection')
plt.xlabel('Feature 1 (Standardized)')
plt.ylabel('Feature 2 (Standardized)')
plt.legend()
plt.show()

# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  # Diagonal line
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

Silhouette Score: 0.531
Davies-Bouldin Index: 0.757
S/DB Ratio: 0.701
Adjusted Rand Index (ARI): 0.201
F1 Score: 0.47
Hubness Mean: 5.0
Hubness Std Dev: 3.898


NameError: name 'outliers_count' is not defined